In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data"

In [3]:
# Define column names based on dataset description
names = ['letter', 'x-box', 'y-box', 'width', 'height', 'onpix', 'x-bar', 'y-bar', 'x2bar', 'y2bar', 'xybar', 'x2ybr', 'xy2br', 'x-ege', 'xegvy', 'y-ege', 'yegvx']


In [4]:
data = pd.read_csv(url,names=names)

In [20]:
data

,letter,x-box,y-box,width,height,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [5]:
#seaparte features and target
X = data.drop('letter',axis=1)
y= data['letter']

In [6]:
#Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y_encoded,test_size=0.2,random_state=42)

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [9]:
#DEfine model
model = Sequential([
    # First hidden layer with 64 neurons and ReLU activation function
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    # Second hidden layer with 64 neurons and ReLU activation function
    Dense(64,activation='relu'),
    # Output layer with neurons equal to the number of classes (26 for letters
    # A-Z) and softmax activation function
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [10]:
#Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
#Train the model
model.fit(X_train,y_train, epochs=20, batch_size=32, validation_split=0.1)

Epoch 1/20
450/450 [==============================] - 8s 12ms/step - loss: 1.8257 - accuracy: 0.4952 - val_loss: 1.1666 - val_accuracy: 0.6825
Epoch 2/20
450/450 [==============================] - 3s 7ms/step - loss: 1.0034 - accuracy: 0.7195 - val_loss: 0.9208 - val_accuracy: 0.7387
Epoch 3/20
450/450 [==============================] - 1s 3ms/step - loss: 0.8346 - accuracy: 0.7660 - val_loss: 0.7927 - val_accuracy: 0.7725
Epoch 4/20
450/450 [==============================] - 2s 4ms/step - loss: 0.7274 - accuracy: 0.7943 - val_loss: 0.6989 - val_accuracy: 0.8000
Epoch 5/20
450/450 [==============================] - 1s 2ms/step - loss: 0.6491 - accuracy: 0.8116 - val_loss: 0.6661 - val_accuracy: 0.8037
Epoch 6/20
450/450 [==============================] - 1s 2ms/step - loss: 0.5905 - accuracy: 0.8283 - val_loss: 0.5631 - val_accuracy: 0.8444
Epoch 7/20
450/450 [==============================] - 1s 3ms/step - loss: 0.5458 - accuracy: 0.8355 - val_loss: 0.5516 - val_accuracy: 0.8394
Epoch

In [12]:
#save the trained model
model.save("letter_recognition_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
#Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy: ',test_acc)

125/125 [==============================] - 0s 2ms/step - loss: 0.3145 - accuracy: 0.8988
Test Accuracy:  0.8987500071525574


In [14]:
#save the label encoder
np.save("label_encoder_classes.npy",label_encoder.classes_)

In [15]:
from tensorflow.keras.models import load_model

In [16]:
# Function to preprocess input dimensions
def preprocess_input(dimensions):
  # Convert dimensions to a numpy array and reshape it to match the input shape
  # of the model
  input_data=np.array(dimensions).reshape(1,-1)
  return input_data

In [17]:
# Function to predict the output using the trained model
def predict_output(model,input_data,label_encoder):
   # Use the model to predict the output
  predicted_class=np.argmax(model.predict(input_data),axis=-1)
  # Convert the predicted class index back to the original letter
  predicted_letter = label_encoder.inverse_transform(predicted_class)
  return predicted_letter[0]

In [18]:
# Load the trained model and label encoder
model=load_model("letter_recognition_model.h5")
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load("label_encoder_classes.npy",allow_pickle=True)

In [24]:
# Input dimensions dynamically from the user
dimension_str = input("Enter the dimensions separated by commas: ")
# Convert the input string to a list of integers
dimensions = list(map(int, dimension_str.split(',')))

Enter the dimensions separated by commas: 2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8


In [25]:
# Preprocess the input dimensions
input_data = preprocess_input(dimensions)

In [ ]:
#2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8

In [26]:
# Predict the output
predicted_letter = predict_output(model, input_data,label_encoder)
print("Predicted Letter: ",predicted_letter)

1/1 [==============================] - 0s 60ms/step
Predicted Letter:  T
